In [5]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()


print('Importing Librairies...')
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style

import pickle
import sys

import xgboost as xgb

import time
import datetime as dt
import os
import shutil
import ib_insync
from ib_insync import *

import smtplib
import mimetypes
from email.message import EmailMessage

print('Librairies imported')
print('')

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5780(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(Fore.YELLOW,Back.BLUE,'ESCAPE GTC Go!em LIVE #25.5 87% -- 0.5 % -- ',Style.RESET_ALL)

print('')

ib = IB()
compte = 'U3229419'
LeNom = 'Live_25'
adjusted = 'oui'
LeChemin = 'DL_2019-12-06/'
choupinet_le_canard_il_est_un_vrai_cafard = LeChemin

#LaDate = input('Entrez la date')
#start = LaDate[:2]+str(int(LaDate[2:4]) - 4)+'-'+LaDate[5:]
#end = LaDate
#stamp = LaDate

print('')

print('adjusted = ',adjusted,', LeNom = ', LeNom,', LeChemin = ',LeChemin)

Importing Librairies...
Librairies imported

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5780(c)
 °0Oo_D.A.G._26_oO0°
  ESCAPE GTC Go!em LIVE #25.5 87% -- 0.5 % --  


adjusted =  oui , LeNom =  Live_25 , LeChemin =  DL_2019-12-06/


In [6]:



def conX():
    util.startLoop()  # uncomment this line when in a notebook
    ib.connect('127.0.0.1', 7497, clientId=1)
    if ib.isConnected() == True:
        print('Golem est maintenant connecté à IB')
    return

def disconX():
    ib.disconnect()
    if ib.isConnected() == False:
        print('Golem est maintenant déconnecté de IB')
    return


def relevpoz():
    CPT = []
    QT = []
    TYPE = []
    STOCK = []
    TICK = []
    AVGCOST = []
    CONID = []
    globalpoz = ib.positions(account=compte)
    
    if globalpoz == []:
        print("Il n'y a aucune position en cours actuellement.")
        poz = pd.DataFrame({'ALERTE': [101]}) 
    else:
        print('Il y a actuellement ',len(globalpoz),'positions en cours')
        for i in range(0,len(globalpoz)):
            CPT.append(globalpoz[i][0])
            TYPE.append(str(globalpoz[i][1]).split("(")[0])
            if str(globalpoz[i][1]).split("(")[0] == 'Future':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            elif str(globalpoz[i][1]).split("(")[0] == 'Forex':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            elif str(globalpoz[i][1]).split("(")[0] == 'Stock':
                TICK.append(str(globalpoz[i][1]).split("'")[1])
            else:
                TICK.append('NA')

            QT.append(globalpoz[i][2])
            AVGCOST.append(globalpoz[i][3])
            CONID.append(ib.positions(account=compte)[i][1].conId)
            poz = pd.DataFrame()
            
            poz['Compte'] = CPT
            poz['Type'] = TYPE
            poz['conId'] = CONID
            poz['Ticker'] = TICK
            poz['Quantité'] = QT
            poz['Average Cost'] = AVGCOST
            #poz.to_csv('Relev_poz/RelevPoz_'+time.strftime("%d_%m_%Y_%H_%M_%S", time.localtime())+'.csv')
    return(poz)

def price_close(LaBase,ticker):
    
    price = round(LaBase[LaBase['Symbol']==ticker].iloc[-1]['Close'],2)
        
    return(price)

def bracketOrder(action: str, quantity: float,limitPrice: float, takeProfitPrice: float,stopLossPrice: float, **kwargs) -> BracketOrder:
    """
    Create a limit order that is bracketed by a take-profit order and
    a stop-loss order. Submit the bracket like:

    .. code-block:: python

        for o in bracket:
            ib.placeOrder(contract, o)

    https://interactivebrokers.github.io/tws-api/bracket_order.html

    Args:
        action: 'BUY' or 'SELL'.
        quantity: Size of order.
        limitPrice: Limit price of entry order.
        takeProfitPrice: Limit price of profit order.
        stopLossPrice: Stop price of loss order.
    """
    assert action in ('BUY', 'SELL')
    reverseAction = 'BUY' if action == 'SELL' else 'SELL'
    parent = LimitOrder(
        action, quantity, limitPrice,
        orderId=ib.client.getReqId(),
        transmit=False,
        **kwargs)
    takeProfit = LimitOrder(
        reverseAction, quantity, takeProfitPrice,
        orderId=ib.client.getReqId(),
        transmit=False,
        parentId=parent.orderId,tif='GTC',
        **kwargs)
    stopLoss = StopOrder(
        reverseAction, quantity, stopLossPrice,
        orderId=ib.client.getReqId(),
        transmit=True,
        parentId=parent.orderId,tif='GTC',
        **kwargs)
    return BracketOrder(parent, takeProfit, stopLoss)


def CLOSE_SHORT(contract,quantity,price,ticker):    
    
    ASSUPPRIMER=[]
    for zz in range(0,len(ib.openTrades())):
        if ib.openTrades()[zz].contract.symbol == ticker :
            ASSUPPRIMER.append(ib.openTrades()[zz].order)
    print(len(ASSUPPRIMER))
    for assupprimer in ASSUPPRIMER:
        ib.cancelOrder(assupprimer)
    
    try:
        transact_price = poz[poz['Ticker'] == ticker].iloc[-1]['Average Cost']
    except:
        transact_price = price
    ib.qualifyContracts(contract)
    order = LimitOrder('BUY', quantity, price,account=compte)
    trade = ib.placeOrder(contract, order)
    print(trade,'\n \n')
    print('Suppression des gtc qui trainent.')
    
    return()

def CLOSE_LONG(contract,quantity,price,ticker):
    
    ASSUPPRIMER=[]
    for zz in range(0,len(ib.openTrades())):
        if ib.openTrades()[zz].contract.symbol == ticker :
            ASSUPPRIMER.append(ib.openTrades()[zz].order)
    print(len(ASSUPPRIMER))
    for assupprimer in ASSUPPRIMER:
        ib.cancelOrder(assupprimer)
    
    try:
        transact_price = poz[poz['Ticker'] == ticker].iloc[-1]['Average Cost']
    except:
        transact_price = price
    ib.qualifyContracts(contract)
    order = LimitOrder('SELL', quantity, price,account=compte)
    trade = ib.placeOrder(contract, order)
    print(trade,'\n \n')
    print('Suppression des gtc qui trainent.')
    
    return()

def OPEN_LONG(contract,quantity,price,ticker):
 
    takeProfitPrice = round(price * 1.02,2)
    stopLossPrice = round(price * 0.85,2)
    ib.qualifyContracts(contract)
    order = bracketOrder(action='BUY', quantity=quantity, limitPrice = price,takeProfitPrice=takeProfitPrice, stopLossPrice=stopLossPrice,account=compte) 
    trade = ib.placeOrder(contract, order.parent)
    trade = ib.placeOrder(contract, order.takeProfit)
    trade = ib.placeOrder(contract, order.stopLoss)
    print(trade,'\n \n')
    return()

def OPEN_SHORT(contract,quantity,price,ticker):

    takeProfitPrice = round(price * 0.96,2)
    stopLossPrice = round(price * 1.15,2)
    ib.qualifyContracts(contract)
    order = bracketOrder(action='SELL',quantity=quantity, limitPrice = price,takeProfitPrice=takeProfitPrice, stopLossPrice=stopLossPrice,account=compte)
    trade = ib.placeOrder(contract, order.parent)
    trade = ib.placeOrder(contract, order.takeProfit)
    trade = ib.placeOrder(contract, order.stopLoss)
    print(trade,'\n \n')
    return()



In [8]:
try:
    disconX()
    ib.sleep(2)
except:
    conX()
    ib.sleep(2)

time.sleep(3)
conX()
time.sleep(3)
if ib.isConnected() == False:
    conX()



Golem est maintenant déconnecté de IB
Golem est maintenant connecté à IB


[Order(orderId=534, clientId=1, permId=1437174952, action='BUY', totalQuantity=92.0, orderType='STP', lmtPrice=0.0, auxPrice=46.0, tif='GTC', ocaGroup='1437174950', ocaType=3, rule80A='0', trailStopPrice=46.0, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U3229419', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True),
 Order(orderId=684, clientId=1, permId=1407384039, action='BUY', totalQuantity=96.0, orderType='LMT', lmtPrice=52.1, auxPrice=0.0, tif='DAY', ocaType=3, rule80A='0', openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U3229419', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True),
 Order(orderId=591, clientId=1, permId=53117740, action='BUY', totalQuantity=107.0, orderType='LMT', lmtPrice=85.0, auxPrice=0.0, tif='GTC', ocaGroup='531

In [10]:
open_orders = ib.openOrders()

In [16]:
open_orders[2]

Order(orderId=591, clientId=1, permId=53117740, action='BUY', totalQuantity=107.0, orderType='LMT', lmtPrice=85.0, auxPrice=0.0, tif='GTC', ocaGroup='53117739', ocaType=3, rule80A='0', openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U3229419', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True)

In [23]:
open_orders[0].orderType,open_orders[0].tif

('STP', 'GTC')

In [17]:
len(open_orders)

95

In [29]:
ii = 10 # ii for input index
ii = (1-(ii/100))/0.96
ii

0.9375000000000001

Peer closed connection


In [26]:
for i in range(0,len(open_orders)):
    if open_orders[i].tif !='GTC':
        continue
    if open_orders[i].orderType !='LMT':
        continue
    if open_orders[i].action !='BUY':
        continue
    

2
4
6
10
12
15
16
22
24
25
28
29
32
35
39
40
41
43
47
50
51
52
53
57
58
63
64
65
66
68
69
72
74
77
79
80
84
86
88
90
93
